In [19]:
# Imports
import pandas as pd
import numpy as np
from data import Database
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import OrdinalEncoder
from torch.utils.data import random_split, TensorDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, ReLU, Dropout
from tensorflow.keras.activations import sigmoid, softmax
from sklearn.preprocessing import StandardScaler

In [20]:
# Connect to Database
db = Database()

In [21]:
# Populate database
db.reset()
db.seed(1000)

'Successfully inserted 1000 documents!'

In [22]:
# Transform data
df = db.dataframe()
enc = OrdinalEncoder()
df["Rarity"] = enc.fit_transform(df[["Rarity"]])

# Split Data

In [23]:
# Create feature matrix and target vector
X = df[["Level", "Health", "Energy", "Sanity"]]
y = df["Rarity"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Establish Baseline
baseline score = 0.31

In [24]:
y_train.value_counts(normalize=True)[0]

0.30875

# Random Forest
best score = 0.99

In [25]:
# Create pipeline
rf_pipe = Pipeline([
    ("scale",StandardScaler()),
    ("clf", RandomForestClassifier())
])

# Param dict for grid search
rf_param_dict = {
    "clf__n_estimators" : [80, 100, 120],
    "clf__min_samples_split" : [2, 4],
    "clf__criterion" : ["gini", "entropy", "log_loss"],
    "clf__min_samples_leaf" : [1, 2]
}

# Instantiate and fit GridSearch
rf_grid = GridSearchCV(rf_pipe, rf_param_dict)
rf_grid.fit(X_train, y_train)
rf_grid.best_estimator_.score(X_test, y_test)

0.99

In [31]:
rf_grid.best_params_

{'clf__criterion': 'gini',
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 2,
 'clf__n_estimators': 80}

# Multinomial Logistic Regression
best score = .69

In [22]:
# Create model
lr_model = LogisticRegressionCV(multi_class="ovr", cv=5, random_state=42)

In [23]:
# Param dict for grid search
grid_params = {
    "Cs" : [8, 10, 12],
    "cv" : [3, 5, 7]
}

# Instantiate and fit grid search
lr_grid = GridSearchCV(lr_model, param_grid=grid_params)
lr_grid.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegressionCV(cv=5, multi_class='ovr',
                                            random_state=42),
             param_grid={'Cs': [8, 10, 12], 'cv': [3, 5, 7]})

In [27]:
lr_grid.score(X_test, y_test)

0.69

# Tensorflow
best score = 1.0

In [28]:
# Build network
tf = Sequential()
tf.add(Dense(10, input_dim=4, activation="relu"))
tf.add(Dense(20, activation="relu")),
tf.add(Dense(10, activation="relu")),
tf.add(Dense(6, activation="softmax"))

stop = EarlyStopping(monitor="loss", patience=3)

# Compile model
tf.compile(loss="sparse_categorical_crossentropy",
           optimizer="nadam",
           metrics=["accuracy"])

In [29]:
# Fit model
tf.fit(X_train, y_train, batch_size=20, epochs=200, callbacks=[stop])

Epoch 1/200
40/40 [==============================] - 2s 3ms/step - loss: 3.8348 - accuracy: 0.1975
Epoch 2/200
40/40 [==============================] - 0s 3ms/step - loss: 1.6061 - accuracy: 0.4363
Epoch 3/200
40/40 [==============================] - 0s 3ms/step - loss: 1.2651 - accuracy: 0.5612
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 1.0905 - accuracy: 0.6338
Epoch 5/200
40/40 [==============================] - 0s 3ms/step - loss: 0.9775 - accuracy: 0.6850
Epoch 6/200
40/40 [==============================] - 0s 3ms/step - loss: 0.8631 - accuracy: 0.7375
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 0.7709 - accuracy: 0.7563
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 0.7053 - accuracy: 0.7912
Epoch 9/200
40/40 [==============================] - 0s 3ms/step - loss: 0.6355 - accuracy: 0.7975
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 0.5835 - accuracy: 0.8125
Epoch 11/

In [30]:
tf.evaluate(X_test, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.0243 - accuracy: 1.0000


[0.02426598034799099, 1.0]

# Conclusion

In this notebook, I trained and evaluated three different classification models, a Random Forest Classifier, a Multinomial Logistic Regression, and a TensorFlow Neural Network. Of these, the Random Forest with an accuracy of .99 and the neural network with an accuracy of 1.0 are the best. I will deploy the Random Forest Classifier since it is the simpler model and still performs at about the same level as the Neural Network.